In [2]:
from bs4 import BeautifulSoup
import requests as requests
import time
import re
from datetime import datetime
import pandas as pd
import numpy as np
from user_modules import scraping_user as su

## Test chunck
NB: need to solve the problem of some items not having a title (these are the comments). At the current time, I solved the problem with a `try - except`.

In [4]:
titles = []

start = time.time()

for i in range(1,11):
    
    try:
        url_posts = 'https://stacker.news/items/%d' % (i)
        response = requests.get(url_posts)
        soup = BeautifulSoup(response.text, 'html.parser')
    
        title = soup.find_all('a', class_='item_title__FH7AS text-reset me-2')
        
        for e in title:
            titles.append(e.get_text())
        
    except:
        continue

end = time.time()

print("The time of execution of above program is :",
      (end-start), "s")

## Collect post header with all the related data

In [1]:
url_posts = 'https://stacker.news/items/1'
response = requests.get(url_posts)
soup = BeautifulSoup(response.text, 'html.parser')

titolo = soup.find('a', class_='item_title__FH7AS text-reset me-2').get_text()
banner = soup.find('div', class_='item_other__MjgP3')

#post_stack = banner.find('span').text

banner_data = []

for i in banner.find_all('span'):
    banner_data.append(i.text)
    
#n_comments = soup.find('a', class_='text-reset position-relative').get_text()
#nym_creator = banner.find('a', ) 

print([titolo, banner_data])


NameError: name 'requests' is not defined

## Collect timestamp of post creation
The following code scans the parsed html response looking for an `<a>` tag that has an attribute `title` matching the timestamp format in the website.
Then it extracts it and turns it into a datetime object. 

In [13]:
# url_posts = 'https://stacker.news/items/1'
# response = requests.get(url_posts)
# soup = BeautifulSoup(response.text, 'html.parser')
# 
# timestamp_pattern = re.compile(r'(\d{4}-\d{2}-\d{2}T\d{2}:\d{2}:\d{2}\.\d{3}Z)')
# original_format = "%Y-%m-%dT%H:%M:%S.%fZ"
# desired_format = "%Y-%m-%d %H:%M:%S"
# 
# timestamp_element = soup.find('a', attrs={'title': timestamp_pattern})
# timestamp_match = timestamp_pattern.search(timestamp_element['title']).group(1)
# 
# timestamp_datetime = datetime.strptime(timestamp_match, original_format)
# 
# print(timestamp_datetime)


2021-06-11 19:26:02.662000


In [22]:
NA = None

def get_item_page(item_code):  # TODO this function could be bundled-in the function scraping_user.get_profile()
    
    # Provided a string returns a bs4.BeautifulSoup object 
    try:
        url_posts = f'https://stacker.news/items/{item_code}'
        response = requests.get(url_posts)
        soup = BeautifulSoup(response.text, 'html.parser')
        return soup
    except:
        return NA
        

## Function that returns the datetime given an html file where a timestamp is present
def get_timedate(item_code):
    
    # Provided a string returns a datetime.datetime object
    page = get_item_page(item_code)
    try:
        timestamp_pattern = re.compile(r'(\d{4}-\d{2}-\d{2}T\d{2}:\d{2}:\d{2}\.\d{3}Z)')
        original_format = "%Y-%m-%dT%H:%M:%S.%fZ"
        
        timestamp_element = page.find('a', attrs={'title': timestamp_pattern})
        timestamp_match = timestamp_pattern.search(timestamp_element['title']).group(1)
        timestamp_datetime = datetime.strptime(timestamp_match, original_format)
        return timestamp_datetime
    except:
        return NA
            

# Scraping Items of stacker.news

In [5]:
df = pd.DataFrame(columns=['titolo', 'numero commenti', 'boost', 'sats', 'betha',"commentors"])
n_posts = range(1,20)

start = time.time()

for number in n_posts:
    # print("Doing", number)
    url_posts = f'https://stacker.news/items/{number}'
    response = requests.get(url_posts)
    soup = BeautifulSoup(response.text, 'html.parser')

    try:
        contenuto=soup.find('div', class_="item_fullItemContainer__ZAYtZ").get_text()
    except:
        continue
    try:
        titolo = soup.find('a', class_='item_title__FH7AS text-reset me-2').get_text()
    except:
        continue
    try:
        banner = soup.find('div', class_='item_other__MjgP3')
    except:
        continue
    try:
        commento = soup.find('a', class_='text-reset position-relative').get_text()
    except:        
        continue
    try:
        a_elements = soup.find_all('a')
    except:
        continue
    
    at_elements = []

    for el in a_elements:
        links = el.get_text()
    
        if links.startswith('@'):
            at_elements.append(el)
    
    commentors_list=[]
    for ind in range(0,len(at_elements)):
        commentors_list.append(at_elements[ind]["href"])


    banner_data = []
    for i in banner.find_all('span'):
        banner_data.append(i.text)
    index=number-1
    for b in banner_data:
        if "boost" in b:
            df.at[index, 'boost'] = b
        if "sats" in b:
            df.at[index, 'sats'] = b
        if "@" in b:
            df.at[index, 'betha'] = b   # TODO: insert the datetime extraction function

    df.at[index, 'titolo'] = titolo
    df.at[index, 'numero commenti'] = commento
    df.at[index, 'contenuto'] = contenuto
    df.at[index, 'commentors'] = commentors_list

end = time.time()

print("The time of execution of above program is :",
      (end-start), "s")

df


/tmp/ipykernel_13131/564396278.py:60: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.at[index, 'contenuto'] = contenuto


The time of execution of above program is : 12.514706373214722 s


,titolo,numero commenti,boost,sats,betha,commentors,contenuto
0,El Salvador Makes Bitcoin Legal Tender,13 comments,4 boost,603 sats,@k00b 11 Jun 2021,"[/k00b, /ln123, /k00b, /k00b, /OneOneSeven, /e...",
1,stacker news soft launch AMA,30 comments,NaN,2 sats,@k00b 11 Jun 2021,"[/k00b, /sha256, /k00b, /banana, /utxoclub, /k...",Figured I'd make a place to raise questions ab...
7,The Lightning Conference 2019: Alex Bosworth,2 comments,NaN,0 sats,@k00b 11 Jun 2021,"[/k00b, /k00b, /k00b, /k00b, /kr, /ekzyis]",


# Scraping user profiles
Users profiles are scraped starting from the list of users extracted by scraping all the items (posts+comments)
The link to get the user profile is `https://stacker.news/$username$` 

## Modularize the code - user profile scraping
Define functions to modularize and simplify the user profile scraping.
The functions are defined in `user_modules/scraping_user.py` and have been tested with `tests/test_scraping_user.py`. To run the tests open a the terminal/CMD and run the script `tests/test_scraping_user.py` from there, running it from a JupyterNotebook could raise errors.
The testing script tested every function in some corner cases (missing data/request error). 

**From the following two approaches we can create the final script for scraping the user profiles**.
The next step would be the creation of a function that loops through the user list and assigns to the rows in the dataframe the values returned from the functions defined in `user_modules/scraping_user.py`.

In [142]:
# TODO: integrate this method into the general scraping script for profile scraping

user_list = np.array(['Monotone',
                      'TNStacker',
                      'kale',
                      'DiracDelta',
                      'kr',
                      'moscowTimeBot',
                      'mpuels',
                      'blockstream_official',
                      'nym',
                      'random_'
                      ])
                      
start = time.time()

for i in np.nditer(user_list):
    print(su.get_total_stacked(i))
    
end = time.time()
print("The average time of execution of the above program for one user is :",
      (end-start)/len(user_list), "s")

123353
53789
357270
181422
1114441
6427
0
50703
164002
1369008
The average time of execution of the above program for one user is : 0.6400596380233765 s


In [145]:
user_list = ['Monotone',
             'TNStacker',
             'kale',
             'DiracDelta',
             'kr',
             'moscowTimeBot',
             'mpuels',
             'blockstream_official',
             'nym',
             'random_',
             ]
                      
start = time.time()

for i in user_list:
    print(su.get_total_stacked(i))
    
end = time.time()
print("The average time of execution of the above program for one user is :",
      (end-start)/len(user_list), "s")

123353
53789
357270
181422
1114441
6427
0
50703
164002
1369008
The average time of execution of the above program for one user is : 0.691546106338501 s
